Divide by two circuit 
=========AUTHORS===========
Dr. Kalenteridis Vasileios 
Batzolis Eleftherios

Python Code Overview
The Python code in this Jupyter notebook builds and places components in the divide-by-2 circuit using the sky130 PDK and gdsfactory layout library.

create_nmos_latch Function: Defines a single NMOS-based latch with eight NMOS transistors. Transistors are arranged in the current mirror, transmission gate, and cross-coupled inverter configurations. The function:

Creates NMOS transistors with specific widths and lengths.
Positions and connects these transistors using both straight and C-routes for proper wiring.
Adds ports for input (D, Dp), clock (CLK, CLKN), output (Q, Qp), and power (VDD, VSS).
create_divide_by_two_circuit Function: Builds the entire divide-by-2 circuit by instantiating two NMOS latches.

Places the second latch below the first latch with adequate separation.
Connects the output of the first latch to the input of the second and vice versa (feedback).
Adds ports for the clock (CLK, CLKN), output (OUT, OUT_B), and power (VDD, VSS).
Connects power ports of both latches to ensure consistent power distribution.
Circuit Display and GDS Export:

After creating the divide-by-two component, display_component shows a scaled view of the component.
The component is written to a GDS file, divide_two.gds, for further physical verification and layout.

In [ ]:
# Setup the environment for the OpenFASOC GDSFactory generator
# You only need to run this block once!

# Clone OpenFASoC

!git clone https://github.com/idea-fasoc/OpenFASOC
#used for testing 
#!git clone https://github.com/Lefteris-B/OpenFASOC

# Install python dependencies
!pip install sky130
!pip install gf180 prettyprinttree svgutils
!pip install gdsfactory==7.7.0

import pathlib
import os

# Install KLayout (via conda)
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}

!bin/micromamba create --yes --prefix $CONDA_PREFIX

# Install from the litex-hub channel
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        klayout

In [ ]:
# Setup the environment for the OpenFASOC GDSFactory generator

# Adding micro-mamba binary directory to the PATH
# This directory contains Klayout
import pathlib
import os
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
# Add conda packages to the PATH
PATH = os.environ['PATH']
%env PATH={PATH}:{CONDA_PREFIX}/bin

%cd /content/OpenFASOC/openfasoc/generators/glayout

In [ ]:
# Intall glayout
%cd /content/OpenFASOC/openfasoc/generators/glayout
!pip install -e .

In [ ]:
# Confirm Instalation
!pip show glayout

In [5]:
import glayout

In [ ]:
%cd /content/OpenFASOC/openfasoc/generators/glayout/glayout/flow/blocks/Divide_by_two_SystemsGenesys
from latch_design import create_nmos_latch_layout
from divide_by_two import create_divide_by_two_circuit
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk as sky130
import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Redirect all outputs here
hide = widgets.Output()

def display_gds(gds_file, scale = 3):
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg('out.svg')
  # Scale the image for displaying
  fig = sg.fromfile('out.svg')
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save('out.svg')

  # Display the image
  IPython.display.display(IPython.display.SVG('out.svg'))

def display_component(component, scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds("out.gds")
  display_gds('out.gds', scale)


In [ ]:
# Create and display the designs
latch = create_nmos_latch_layout(sky130)
display_component(latch, scale=3)

divider = create_divide_by_two_circuit(sky130)
display_component(divider, scale=3)

# Save GDS files
latch.write_gds("latch.gds")
divider.write_gds("divide_two.gds")